In [5]:
!nvidia-smi

Wed May  8 13:14:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1D.0 Off |                    0 |
|  0%   28C    P8              15W / 300W |      0MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import os
import torch
import torch.nn.functional as F

from dataclasses import asdict
from transformers import RobertaTokenizer
from torch.utils.data import Dataset, DataLoader, random_split
#from pl_bolts.optimizers import LinearWarmupCosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

from config import Config
from data.ee import EE
from data.bde import BDE
from data.drugs import Drugs
from data.kraken import Kraken
from happy_config import ConfigLoader
from utils.early_stopping import EarlyStopping, generate_checkpoint_filename

from models.model_1d import LSTM, Transformer
from models.models_1d.utils import construct_fingerprint, construct_smiles, concatenate_smiles
#from train_1d import *

ModuleNotFoundError: No module named 'torch_geometric'